In [110]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/home/thalvadzhiev_gmail_com/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12044853500389275560
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11270098125
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 18338255659155598178
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 11285974221
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 12333419102297717280
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:00:05.0, compute capability: 3.7"
]


In [2]:
import sys

import sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import keras
import pickle

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras import Model
import cv2, numpy as np

from PIL import Image

%matplotlib inline

In [3]:
with open("faces_dataset_conv.pickle", "rb") as f:
    data = pickle.load(f)
    

In [4]:
train_ds = data["train_dataset"]
train_labels = data["train_labels"]
pub_test_ds = data["public_test"]
pub_test_labels = data["public_test_labels"]
priv_test_ds = data["private_test"]
priv_test_labels = data["private_test_labels"]

In [5]:
train_ds_res = train_ds.reshape(train_ds.shape[0], 48, 48)
pub_test_ds_res = pub_test_ds.reshape(pub_test_ds.shape[0], 48, 48)
priv_test_ds_res = priv_test_ds.reshape(priv_test_ds.shape[0], 48, 48)

In [6]:
train_labels = np_utils.to_categorical(train_labels, 7)
pub_test_labels = np_utils.to_categorical(pub_test_labels, 7)
priv_test_labels = np_utils.to_categorical(priv_test_labels, 7)

In [7]:
!pip install keras_vggface

    100% |████████████████████████████████| 49.1MB 895kB/s eta 0:00:01
  Running setup.py bdist_wheel for keras-vggface ... done
  Stored in directory: /home/thalvadzhiev_gmail_com/.cache/pip/wheels/72/aa/01/eb7baeb2f6e2d2f0d2aabddb5f01d57fa22fbd019ee2799bf5
Successfully built keras-vggface
distributed 1.21.8 requires msgpack, which is not installed.


In [7]:
s=224
from keras_vggface.vggface import VGGFace
modelReady = VGGFace(include_top=True, input_shape=(s,s,3))

In [23]:
modelReady.layers.pop()
modelReady.layers.pop()
##remove last two layers
modelReady.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [8]:
def resize_images(images,size=(s,s)):
  resized=[]
  for im in images:
    resized.append(np.array(Image.fromarray(np.uint8(cv2.cvtColor(im, cv2.COLOR_GRAY2RGB) * 255) , 'RGB').resize(size)).reshape(*size,3))
    
  return np.array(resized)    
  

In [9]:
resized_training = resize_images(train_ds_res)
resized_pub_test = resize_images(pub_test_ds_res)
resized_priv_test = resize_images(priv_test_ds_res)

In [10]:
resized_train_big = np.vstack([resized_training, resized_priv_test])
resized_train_labels_big = np.concatenate([train_labels, priv_test_labels])

In [0]:
# layer_count = 0
# for layer in custom_vgg_model.layers:
# 	layer_count = layer_count+1
# for l in range(layer_count-3):
# 	modelReady.layers[l].trainable=False

In [24]:
for layer in modelReady.layers:
  layer.trainable=False


In [25]:
#custom parameters
nb_class = 7
hidden_dim = 512

last_layer = modelReady.get_layer('fc7/relu').output
# x = Flatten(name='flatten')(last_layer)
# x = Dense(hidden_dim, activation='relu', name='fc6')(x)
# x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(last_layer)
custom_vgg_model = Model(modelReady.input, out)

In [26]:
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [11]:
from keras.callbacks import ModelCheckpoint

In [7]:
opt = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
checkpoint = ModelCheckpoint('weights_emotion_vgg.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
                        save_weights_only=True, mode='max')

In [12]:
from keras.utils import multi_gpu_model

In [28]:
parallel_model = multi_gpu_model(custom_vgg_model, gpus=2)
parallel_model.compile(optimizer="adam", loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
parallel_model.fit(resized_train_big, resized_train_labels_big, 
          batch_size=64, epochs=3, verbose=1, callbacks=[checkpoint], validation_data=[resized_pub_test, pub_test_labels] )

Train on 32298 samples, validate on 3589 samples
Epoch 1/3
32298/32298 [==============================] - 182s 6ms/step - loss: 2.3438 - acc: 0.4812 - val_loss: 2.0703 - val_acc: 0.4993

Epoch 00001: val_acc improved from -inf to 0.49930, saving model to weights_emotion_vgg.hdf5
Epoch 2/3
32298/32298 [==============================] - 177s 5ms/step - loss: 1.6124 - acc: 0.5743 - val_loss: 2.0299 - val_acc: 0.5336

Epoch 00002: val_acc improved from 0.49930 to 0.53357, saving model to weights_emotion_vgg.hdf5
Epoch 3/3
32298/32298 [==============================] - 177s 5ms/step - loss: 1.4418 - acc: 0.6072 - val_loss: 2.0163 - val_acc: 0.5266

Epoch 00003: val_acc did not improve from 0.53357


In [13]:
patience=50
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [14]:
early_stop = EarlyStopping('val_acc', patience=patience)
reduce_lr = ReduceLROnPlateau('val_acc', factor=0.1,
                                  patience=int(patience/4), verbose=1)

In [38]:
parallel_model.fit(resized_train_big, resized_train_labels_big, 
          batch_size=64, epochs=20, verbose=1, callbacks=[checkpoint, early_stop, reduce_lr], validation_data=[resized_pub_test, pub_test_labels] )

Train on 32298 samples, validate on 3589 samples
Epoch 1/20
32298/32298 [==============================] - 176s 5ms/step - loss: 1.3495 - acc: 0.6247 - val_loss: 2.1011 - val_acc: 0.5339

Epoch 00001: val_acc improved from 0.53357 to 0.53385, saving model to weights_emotion_vgg.hdf5
Epoch 2/20
32298/32298 [==============================] - 177s 5ms/step - loss: 1.2943 - acc: 0.6412 - val_loss: 2.1444 - val_acc: 0.5272

Epoch 00002: val_acc did not improve from 0.53385
Epoch 3/20
32298/32298 [==============================] - 176s 5ms/step - loss: 1.2828 - acc: 0.6494 - val_loss: 2.2644 - val_acc: 0.5383

Epoch 00003: val_acc improved from 0.53385 to 0.53831, saving model to weights_emotion_vgg.hdf5
Epoch 4/20
32298/32298 [==============================] - 176s 5ms/step - loss: 1.2558 - acc: 0.6542 - val_loss: 2.2735 - val_acc: 0.5472

Epoch 00004: val_acc improved from 0.53831 to 0.54723, saving model to weights_emotion_vgg.hdf5
Epoch 5/20
32298/32298 [==============================] -

In [11]:
s=224
from keras_vggface.vggface import VGGFace
model2 = VGGFace(include_top=True, input_shape=(s,s,3))

In [12]:
model2.layers.pop()
model2.layers.pop()
# modelReady.layers.pop()
##remove last two layers
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [56]:
for layer in range(len(model2.layers) -2 ):
    model2.layers[layer].trainable = False

In [57]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [13]:
nb_class=7
last_layer = model2.get_layer('fc7/relu').output
# x = Flatten(name='flatten')(last_layer)
# x = Dense(hidden_dim, activation='relu', name='fc6')(x)
# x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(last_layer)
model_fc6_trainable = Model(model2.input, out)

In [14]:
model_fc6_trainable.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [15]:
parallel_model_2 = multi_gpu_model(model_fc6_trainable, gpus=2)
opt = SGD(lr=0.0001, decay=0.0005, momentum=0.9, nesterov=True)

parallel_model_2.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
checkpoint2 = ModelCheckpoint('weights_emotion_vgg_all_layer_trainable.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
                        save_weights_only=True, mode='max')

In [17]:
parallel_model_2.load_weights("weights_emotion_vgg_all_layer_trainable.hdf5")

In [22]:
parallel_model_2.fit(resized_train_big, resized_train_labels_big, 
          batch_size=64, epochs=30, verbose=1, callbacks=[checkpoint2, early_stop, reduce_lr], validation_data=[resized_pub_test, pub_test_labels] )

Train on 32298 samples, validate on 3589 samples
Epoch 1/30
32298/32298 [==============================] - 544s 17ms/step - loss: 0.2951 - acc: 0.8998 - val_loss: 1.3312 - val_acc: 0.6590

Epoch 00001: val_acc improved from -inf to 0.65896, saving model to weights_emotion_vgg_all_layer_trainable.hdf5
Epoch 2/30
32298/32298 [==============================] - 528s 16ms/step - loss: 0.1727 - acc: 0.9471 - val_loss: 1.5586 - val_acc: 0.6668

Epoch 00002: val_acc improved from 0.65896 to 0.66676, saving model to weights_emotion_vgg_all_layer_trainable.hdf5
Epoch 3/30
32298/32298 [==============================] - 528s 16ms/step - loss: 0.1018 - acc: 0.9741 - val_loss: 1.6661 - val_acc: 0.6670

Epoch 00003: val_acc improved from 0.66676 to 0.66704, saving model to weights_emotion_vgg_all_layer_trainable.hdf5
Epoch 4/30
32298/32298 [==============================] - 529s 16ms/step - loss: 0.0672 - acc: 0.9845 - val_loss: 1.8416 - val_acc: 0.6578

Epoch 00004: val_acc did not improve from 0.66

KeyboardInterrupt: 

## Run again from scratch

In [18]:
s=224
from keras_vggface.vggface import VGGFace
model3 = VGGFace(include_top=True, input_shape=(s,s,3))
model3.layers.pop()
model3.layers.pop()
# modelReady.layers.pop()
##remove last two layers
# model3.summary()
nb_class=7
last_layer = model3.get_layer('fc7/relu').output
# x = Flatten(name='flatten')(last_layer)
# x = Dense(hidden_dim, activation='relu', name='fc6')(x)
# x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(last_layer)
model_fc7_trainable2 = Model(model3.input, out)
model_fc7_trainable2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [19]:
parallel_model_3 = multi_gpu_model(model_fc7_trainable2, gpus=2)
opt = SGD(lr=0.0001, decay=0.0005, momentum=0.9, nesterov=True)

parallel_model_3.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
checkpoint3 = ModelCheckpoint('weights_emotion_vgg_all_layer_trainable_new--{val_loss:.2f}--{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
                        save_weights_only=True, mode='max')

In [ ]:
parallel_model_3.fit(resized_training, train_labels, 
          batch_size=64, epochs=40, verbose=1, callbacks=[checkpoint3, early_stop], validation_data=[resized_pub_test, pub_test_labels] )

Train on 28709 samples, validate on 3589 samples
Epoch 1/40
28709/28709 [==============================] - 497s 17ms/step - loss: 1.3396 - acc: 0.5138 - val_loss: 1.1689 - val_acc: 0.5698

Epoch 00001: val_acc improved from -inf to 0.56980, saving model to weights_emotion_vgg_all_layer_trainable_new--1.17--0.5698.hdf5
Epoch 2/40
28709/28709 [==============================] - 487s 17ms/step - loss: 1.0250 - acc: 0.6212 - val_loss: 1.0924 - val_acc: 0.5957

Epoch 00002: val_acc improved from 0.56980 to 0.59571, saving model to weights_emotion_vgg_all_layer_trainable_new--1.09--0.5957.hdf5
Epoch 3/40
28709/28709 [==============================] - 486s 17ms/step - loss: 0.9029 - acc: 0.6721 - val_loss: 1.0493 - val_acc: 0.6172

Epoch 00003: val_acc improved from 0.59571 to 0.61716, saving model to weights_emotion_vgg_all_layer_trainable_new--1.05--0.6172.hdf5
Epoch 4/40
28709/28709 [==============================] - 489s 17ms/step - loss: 0.8125 - acc: 0.7048 - val_loss: 1.0430 - val_acc: 

In [24]:
parallel_model_3.load_weights("weights_emotion_vgg_all_layer_trainable.hdf5")

In [27]:
parallel_model_3.evaluate(resized_pub_test, pub_test_labels, batch_size=64)

3589/3589 [==============================] - 20s 5ms/step


[2.0736898142967957, 0.6703817219447212]

In [0]:
del priv_test_ds_res

In [49]:
# model2.save_weights("best_weights_model2.hdf5")
# model2.load_weights("best_weights_model2.hdf5")

NameError: ignored

## Just FC layers train

In [15]:
s=224
from keras_vggface.vggface import VGGFace
model4 = VGGFace(include_top=True, input_shape=(s,s,3))
model4.layers.pop()
model4.layers.pop()
# modelReady.layers.pop()
##remove last two layers
# model3.summary()
for layer in range(len(model4.layers) -4 ):
    model4.layers[layer].trainable = False
nb_class=7
last_layer = model4.get_layer('fc7/relu').output
# x = Flatten(name='flatten')(last_layer)
# x = Dense(hidden_dim, activation='relu', name='fc6')(x)
# x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(last_layer)
model_fc7_trainable_fc = Model(model4.input, out)
model_fc7_trainable_fc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [16]:
parallel_model_4 = multi_gpu_model(model_fc7_trainable_fc, gpus=2)
opt = SGD(lr=0.0001, decay=0.0005, momentum=0.9, nesterov=True)

parallel_model_4.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
checkpoint5 = ModelCheckpoint('weights_emotion_vgg_fc_trainable_new--{val_loss:.2f}--{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
                        save_weights_only=True, mode='max')

In [ ]:
parallel_model_4.fit(resized_training, train_labels, 
          batch_size=64, epochs=20, verbose=1, callbacks=[checkpoint5, early_stop], validation_data=[resized_pub_test, pub_test_labels] )

Train on 28709 samples, validate on 3589 samples
Epoch 1/20
28709/28709 [==============================] - 241s 8ms/step - loss: 1.7881 - acc: 0.4848 - val_loss: 1.3529 - val_acc: 0.5405

Epoch 00001: val_acc improved from -inf to 0.54054, saving model to weights_emotion_vgg_fc_trainable_new--1.35--0.5405.hdf5
Epoch 2/20
28709/28709 [==============================] - 237s 8ms/step - loss: 0.9869 - acc: 0.6520 - val_loss: 1.2662 - val_acc: 0.5773

Epoch 00002: val_acc improved from 0.54054 to 0.57732, saving model to weights_emotion_vgg_fc_trainable_new--1.27--0.5773.hdf5
Epoch 3/20
28709/28709 [==============================] - 236s 8ms/step - loss: 0.7773 - acc: 0.7318 - val_loss: 1.2277 - val_acc: 0.5890

Epoch 00003: val_acc improved from 0.57732 to 0.58902, saving model to weights_emotion_vgg_fc_trainable_new--1.23--0.5890.hdf5
Epoch 4/20
28709/28709 [==============================] - 237s 8ms/step - loss: 0.6507 - acc: 0.7845 - val_loss: 1.2090 - val_acc: 0.5960

Epoch 00004: val_

## No init

In [15]:
s=224
from keras_vggface.vggface import VGGFace
model5= VGGFace(include_top=True, input_shape=(s,s,3), weights=None)
model5.layers.pop()
model5.layers.pop()
# modelReady.layers.pop()
##remove last two layers
# # model3.summary()
# for layer in range(len(model4.layers) -4 ):
#     model4.layers[layer].trainable = False
nb_class=7
last_layer = model5.get_layer('fc7/relu').output
# x = Flatten(name='flatten')(last_layer)
# x = Dense(hidden_dim, activation='relu', name='fc6')(x)
# x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(last_layer)
model_fc7_trainable_fc = Model(model5.input, out)
model_fc7_trainable_fc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [16]:
parallel_model_5 = multi_gpu_model(model_fc7_trainable_fc, gpus=2)
opt = SGD(lr=0.0001, decay=0.0005, momentum=0.9, nesterov=True)

parallel_model_5.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
checkpoint6 = ModelCheckpoint('weights_emotion_vgg_all_no_init--{val_loss:.2f}--{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True,
                        save_weights_only=True, mode='max')

In [18]:
parallel_model_5.fit(resized_training, train_labels, 
          batch_size=64, epochs=20, verbose=1, callbacks=[checkpoint6, early_stop], validation_data=[resized_pub_test, pub_test_labels] )

Train on 28709 samples, validate on 3589 samples
Epoch 1/20
28709/28709 [==============================] - 484s 17ms/step - loss: 1.7668 - acc: 0.2852 - val_loss: 1.7308 - val_acc: 0.2990

Epoch 00001: val_acc improved from -inf to 0.29897, saving model to weights_emotion_vgg_all_no_init--1.73--0.2990.hdf5
Epoch 2/20
28709/28709 [==============================] - 470s 16ms/step - loss: 1.6407 - acc: 0.3666 - val_loss: 1.6157 - val_acc: 0.3826

Epoch 00002: val_acc improved from 0.29897 to 0.38256, saving model to weights_emotion_vgg_all_no_init--1.62--0.3826.hdf5
Epoch 3/20
28709/28709 [==============================] - 468s 16ms/step - loss: 1.5641 - acc: 0.4027 - val_loss: 1.5913 - val_acc: 0.3870

Epoch 00003: val_acc improved from 0.38256 to 0.38702, saving model to weights_emotion_vgg_all_no_init--1.59--0.3870.hdf5
Epoch 4/20
28709/28709 [==============================] - 469s 16ms/step - loss: 1.5063 - acc: 0.4283 - val_loss: 1.5176 - val_acc: 0.4280

Epoch 00004: val_acc improve